In [1]:
import os
import modules.util as util
import math
import numpy as np
import modules.datagenerator as dtgen
import modules.network as network
import modules.loss as loss
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from tensorflow import keras

2023-03-08 19:58:27.797946: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
home=os.path.abspath(os.getcwd())
data_path=os.path.join(home, 'data_testset')

#global data
pairs=[]
classes=[]

for file in os.listdir(data_path):
    classes.append(file)

In [3]:
util.make_pairs(data_path, pairs, classes)

# partition = {'train': np.arange(math.floor(len(pairs)*.6)),
#              'validation': np.arange(math.floor(len(pairs)*.6),math.floor(len(pairs)*.8)),
#              'test': np.arange(math.floor(len(pairs)*.8), math.floor(len(pairs)))}

partition = {'test': np.arange(math.floor(len(pairs)))}

# Generators
test_generator = dtgen.DataGenerator(partition['test'], pairs, batch_size=1)

In [4]:
print(f"Number of Test Pairs: {len(partition['test'])}")

Number of Test Pairs: 1320


In [5]:
siamese_obj = network.SiameseNetwork()
siamese_network = siamese_obj.make_siamese_net()

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [6]:
siamese_network.compile(loss= loss.loss(1), optimizer=tf.keras.optimizers.Adam(learning_rate=0.5e-3), # 0.5e-6
                    metrics=["accuracy"])

In [7]:
# Re-evaluate the model
loss, acc = siamese_network.evaluate(test_generator, verbose=2)
print("Unrestored model, accuracy: {:5.2f}%".format(100 * acc))


1320/1320 - 76s - loss: 0.2567 - accuracy: 0.5069 - 76s/epoch - 58ms/step
Unrestored model, accuracy: 50.69%


In [8]:
checkpoint_path = os.path.join(home, 'checkpoints')

# Loads the weights
siamese_network.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = siamese_network.evaluate(test_generator, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

2023-03-08 20:00:21.584216: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open /home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/full_data_6hrs/checkpoints: FAILED_PRECONDITION: /home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/full_data_6hrs/checkpoints; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


1320/1320 - 65s - loss: 0.1957 - accuracy: 0.7029 - 65s/epoch - 50ms/step
Restored model, accuracy: 70.29%


In [36]:
siamese_network.summary()

Model: "siamese_network"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 pixel_values1 (InputLayer)     [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 pixel_values2 (InputLayer)     [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 permute (Permute)              (None, 3, 224, 224)  0           ['pixel_values1[0][0]']          
                                                                                    

In [38]:
siamese_network.output_shape

(None, 197, 1)

: 

In [9]:
test_ls=siamese_network.predict(test_generator)

1320/1320 [==============================] - 73s 51ms/step


In [10]:
test_ls.shape


(1320, 197, 1)

In [12]:
test_ls[0].shape

(197, 1)

In [37]:
test_ls[0][0]

array([0.07245867], dtype=float32)

In [13]:
test_ls[0]

array([[0.07245867],
       [0.15490232],
       [0.16010123],
       [0.16305007],
       [0.14026809],
       [0.13992535],
       [0.14557225],
       [0.13010198],
       [0.1666094 ],
       [0.14372355],
       [0.13941506],
       [0.13695613],
       [0.1344357 ],
       [0.1565521 ],
       [0.15733507],
       [0.13463421],
       [0.12083973],
       [0.16401783],
       [0.14322169],
       [0.14614454],
       [0.15116133],
       [0.14298269],
       [0.16900666],
       [0.15653174],
       [0.15090202],
       [0.15311497],
       [0.14450806],
       [0.16431348],
       [0.1484355 ],
       [0.11601086],
       [0.12294892],
       [0.16484076],
       [0.13738506],
       [0.10561806],
       [0.11932145],
       [0.11798118],
       [0.12567224],
       [0.1570328 ],
       [0.12782711],
       [0.1408458 ],
       [0.12994456],
       [0.16359584],
       [0.15251386],
       [0.11085621],
       [0.12536943],
       [0.13644382],
       [0.14879836],
       [0.104

In [11]:
test_ls

array([[[0.07245867],
        [0.15490232],
        [0.16010123],
        ...,
        [0.15452226],
        [0.12400997],
        [0.1298111 ]],

       [[0.32431805],
        [0.31230256],
        [0.30389175],
        ...,
        [0.25514022],
        [0.29920828],
        [0.3016249 ]],

       [[0.9718195 ],
        [0.98061484],
        [0.9797039 ],
        ...,
        [0.9798803 ],
        [0.97854596],
        [0.9827193 ]],

       ...,

       [[0.3480976 ],
        [0.28238043],
        [0.29814732],
        ...,
        [0.31118485],
        [0.26301304],
        [0.36756113]],

       [[0.9946181 ],
        [0.99776304],
        [0.99770397],
        ...,
        [0.99783295],
        [0.997615  ],
        [0.99785256]],

       [[0.3519332 ],
        [0.35814458],
        [0.4056054 ],
        ...,
        [0.38512814],
        [0.4033972 ],
        [0.3827549 ]]], dtype=float32)

In [62]:
len(test_generator)

1304

In [16]:
label=[]
for i in range(0,len(test_ls)):
    label+=[test_generator.labels[i][2]]
print(len(label))

1320


In [14]:
labels = []
for i in test_generator.labels:
    # print(i[2])
    labels.append(i[2])




In [27]:
test_generator.labels[3]

['/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/full_data_6hrs/data_testset/W-048/W-048-12.bmp',
 '/home/halim/Documents/ECE613/GitHub/Face_id_meta/canada_compute_results/full_data_6hrs/data_testset/W-044/W-044-13.bmp',
 1,
 224,
 70]

In [35]:
len(test_generator)

1320

In [15]:
print(len(labels))

1320


In [19]:
labels[0]

0

In [31]:
test_one=siamese_network.predict([test_generator[3]])

ValueError: in user code:

    File "/home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/home/halim/anaconda3/envs/ViT-setup/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/halim/anaconda3/envs/ViT-setup/lib/py

In [66]:
pred_lbl=[]
pred_lbl = test_ls>=0.5 
pred_lbl[0][0]

array([False])

In [67]:
wrong_ls=[]
wrong_ls = pred_lbl!=label
